# Richter's Predictor: Modeling Earthquake Damage

Source: https://www.drivendata.org/competitions/57/nepal-earthquake/

Based on aspects of building location and construction, your goal is to predict the level of damage to buildings caused by the 2015 Gorkha earthquake in Nepal.

EVALUATION METRIC
Fmicro=2⋅Pmicro⋅Rmicro/Pmicro+Rmicro
The metric used for this competition is the micro-averaged F1 score.

In [16]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier,ExtraTreesClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

# Get data from website

In [ ]:
!wget https://s3.amazonaws.com/drivendata/data/57/public/train_values.csv -nc -P ./nepal
!wget https://s3.amazonaws.com/drivendata/data/57/public/train_labels.csv -nc -P ./nepal
!wget https://s3.amazonaws.com/drivendata/data/57/public/test_values.csv  -nc -P ./nepal

# Import Data

In [2]:
dtypes = {'geo_level_1_id': 'object', 
          'geo_level_2_id' : 'object', 
          'geo_level_3_id' : 'object'}
X = pd.read_csv('./nepal/train_values.csv', index_col= 'building_id',
               dtype =dtypes )

In [6]:
X.head()

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,...,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
building_id,,,,,,,,,,,,,,,,,,,,,
802906,6,487,12198,2,30,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,0
28830,8,900,2812,2,10,8,7,o,r,n,...,0,0,0,0,0,0,0,0,0,0
94947,21,363,8973,2,10,5,5,t,r,n,...,0,0,0,0,0,0,0,0,0,0
590882,22,418,10694,2,10,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,0
201944,11,131,1488,3,30,8,9,t,r,n,...,0,0,0,0,0,0,0,0,0,0


In [7]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 260601 entries, 802906 to 747594
Data columns (total 38 columns):
geo_level_1_id                            260601 non-null object
geo_level_2_id                            260601 non-null object
geo_level_3_id                            260601 non-null object
count_floors_pre_eq                       260601 non-null int64
age                                       260601 non-null int64
area_percentage                           260601 non-null int64
height_percentage                         260601 non-null int64
land_surface_condition                    260601 non-null object
foundation_type                           260601 non-null object
roof_type                                 260601 non-null object
ground_floor_type                         260601 non-null object
other_floor_type                          260601 non-null object
position                                  260601 non-null object
plan_configuration                        2

In [3]:
y = pd.read_csv('./nepal/train_labels.csv', index_col = 'building_id')['damage_grade']

## Attempt 1: Model w/ One Feature

In [6]:
X_height = X[['height_percentage']]
X_height.head()

,height_percentage
building_id,
802906,5
28830,7
94947,5
590882,5
201944,9


In [23]:
X_train, X_test, y_train, y_test = train_test_split(X_height,
                                                    y,
                                                    test_size=0.2,
                                                    random_state =42)

In [9]:
one_feat_model = LogisticRegression(solver='lbfgs', multi_class = 'auto')
one_feat_model.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [10]:
y_train_pred = one_feat_model.predict(X_train)

In [11]:
print('In-sample f1 score:')
f1_score(y_train, y_train_pred, average ='micro')

In-sample f1 score:


0.5699779355333845

In [12]:
y_test_pred = one_feat_model.predict(X_test)

In [13]:
print('Out-sample f1 score:')
f1_score(y_test, y_test_pred, average ='micro')

Out-sample f1 score:


0.5660290477926364

In [24]:
X_train.shape

(208480, 1)

# Create Submission file

In [14]:
X_comp_test = pd.read_csv('./nepal/test_values.csv', index_col = 'building_id')

In [15]:
y_comp_pred = one_feat_model.predict(X_comp_test[['height_percentage']])

In [16]:
y_submission = pd.DataFrame(y_comp_pred, index= X_comp_test.index, columns = ['damage_grade'])

In [17]:
y_submission.to_csv('nepal/1st_sub.csv')

Score:0.56

# Create function for repetitive tasks

### Function for calculating insample and out sample score

In [4]:
def get_metrics(model, X_train, X_test, y_train, y_test):
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    in_samp_score = f1_score(y_train, y_train_pred, average= 'micro')
    out_samp_score = f1_score(y_test, y_test_pred, average='micro')
    print('In-sample f1 score:',in_samp_score)
    print('Out-sample f1 score:', out_samp_score)

### Function for creating submission file

In [5]:
def create_submission(model, X_cols=None):
    X= pd.read_csv('./nepal/test_values.csv', index_col = 'building_id',
                  dtype = {'geo_level_1_id': 'object', 
                           'geo_level_2_id' : 'object',
                           'geo_level_3_id' : 'object'})
    if X_cols != None:
        X =X[X_cols]
    y_pred = model.predict(X)
    submission = pd.DataFrame(y_pred, index=X.index,
                             columns = ['damage_grade'])
    date_string =pd.Timestamp.utcnow().strftime(format='%Y-%m-%d_%H%M_')
    submission.to_csv(f'./nepal/{date_string}submission.csv')

In [17]:
get_metrics(one_feat_model, X_train, X_test, y_train, y_test)

In-sample f1 score: 0.5699779355333845
Out-sample f1 score: 0.5660290477926364


In [20]:
create_submission(one_feat_model, ['height_percentage'])

# #Attempt2: Model w/ All Numerical Features

In [4]:
##Columns with numerical data type
X.select_dtypes(include='number').info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 260601 entries, 802906 to 747594
Data columns (total 27 columns):
count_floors_pre_eq                       260601 non-null int64
age                                       260601 non-null int64
area_percentage                           260601 non-null int64
height_percentage                         260601 non-null int64
has_superstructure_adobe_mud              260601 non-null int64
has_superstructure_mud_mortar_stone       260601 non-null int64
has_superstructure_stone_flag             260601 non-null int64
has_superstructure_cement_mortar_stone    260601 non-null int64
has_superstructure_mud_mortar_brick       260601 non-null int64
has_superstructure_cement_mortar_brick    260601 non-null int64
has_superstructure_timber                 260601 non-null int64
has_superstructure_bamboo                 260601 non-null int64
has_superstructure_rc_non_engineered      260601 non-null int64
has_superstructure_rc_engineered          260601 non

In [5]:
##Another way : Columns with numerical data type
numerical_features = [col for col in X.columns if X[col].dtype=='int64']

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.2,
                                                    random_state =42)

In [7]:
X_train.shape

(208480, 38)

In [8]:
num_feat_model = LogisticRegression(solver ='lbfgs',max_iter =200,
                                    multi_class='auto')

In [9]:
num_feat_model.fit(X_train[numerical_features], y_train)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=200,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [14]:
get_metrics(num_feat_model,X_train[numerical_features], X_test[numerical_features], y_train, y_test )

In-sample f1 score: 0.5736713353798926
Out-sample f1 score: 0.5724947717810479


# Attempt3: Numerical Features w/ Another Predictor

In [16]:
xg_num_model = GradientBoostingClassifier()

xg_num_model.fit(X_train[numerical_features], y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='auto',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [17]:
get_metrics(xg_num_model,X_train[numerical_features], X_test[numerical_features], y_train, y_test )

In-sample f1 score: 0.5924501151189563
Out-sample f1 score: 0.5903762399033019


In [ ]:
create_submission(xg_num_model,numerical_features)

Score : '0.5893'

In [18]:
rfc_num_model = RandomForestClassifier()

rfc_num_model.fit(X_train[numerical_features], y_train)
y_train_pred = rfc_num_model.predict(X_train[numerical_features])

print('In-sample f1 score:')
f1_score(y_train, y_train_pred, average='micro')

/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In-sample f1 score:


0.7059238296239447

In [19]:
get_metrics(rfc_num_model,X_train[numerical_features], X_test[numerical_features], y_train, y_test )

In-sample f1 score: 0.7059238296239447
Out-sample f1 score: 0.569674411465628


In [45]:
create_submission(rfc_num_model,numerical_features )

In [ ]:
Score: '0.6559' rank:483

# Attemp 4:  Use all features(numeric and categorical)

In [8]:
categorical_variables = [ col for col in X.columns
                         if X[col].dtype == 'object']


In [9]:
ct = ColumnTransformer([
    ('ohe', OneHotEncoder(handle_unknown = 'ignore'),categorical_variables)],
    remainder = 'passthrough')

all_feat_model = Pipeline([
    ('transformer', ct),
    #('classifier', LogisticRegression(solver= 'lbfgs', multi_class='auto'))
    ('classifier', RandomForestClassifier())
])

In [ ]:
###Onehot encoder is increasing the features to a great deal

In [10]:
all_feat_model.fit(X_train,y_train)  ##So that All category should be included

/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Pipeline(memory=None,
         steps=[('transformer',
                 ColumnTransformer(n_jobs=None, remainder='passthrough',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('ohe',
                                                  OneHotEncoder(categorical_features=None,
                                                                categories=None,
                                                                drop=None,
                                                                dtype=<class 'numpy.float64'>,
                                                                handle_unknown='ignore',
                                                                n_values=None,
                                                                sparse=True),
                                                  ['geo_level_1_id',
                                         

In [12]:
get_metrics(all_feat_model, X_train, X_test, y_train, y_test )

In-sample f1 score: 0.9696277820414428
Out-sample f1 score: 0.7113639415974368


In [13]:
create_submission(all_feat_model)

Score:'0.6559' with Logistic Regression. 
Score:'0.7171' with Random Forest Classifier(rank372/1831)

### Attempt 5: Ensemble Predictor with Grid Search

In [ ]:
##PCA and TruncatedSVD is overloading the kernel. Also the performance is not a big help

In [20]:
ct = ColumnTransformer([
    ('ohe', OneHotEncoder(handle_unknown = 'ignore'),categorical_variables)],
    remainder = 'passthrough')

param_grid = {#'max_depth': range(3,13,3),
             'min_samples_split':np.power(2,np.arange(1,6)),
             'min_samples_leaf':np.power(2,np.arange(1,6))}

gs = GridSearchCV(ExtraTreesClassifier(),
                 param_grid = param_grid,
                 cv = 3,
                 n_jobs = 6,
                 verbose=1)

all_feat_model1 = Pipeline([
    ('transformer', ct),
    ('classifier', gs)
])

In [ ]:
all_feat_model1.fit(X_train, y_train)

In [ ]:
get_metrics(all_feat_model1, X_train, X_test, y_train, y_test )